# Merge CallSign, CMA and Location

In [2]:
import pandas as pd
import re

In [1]:
import csv
import os

## References

`re.`:
* https://medium.com/factory-mind/regex-tutorial-a-simple-cheatsheet-by-examples-649dc1c3f285
* https://docs.python.org/3/library/re.
* https://docs.python.org/3/howto/regex.html#regex-howto
* https://www.journaldev.com/23763/python-remove-spaces-from-string

`csv.`:
* https://docs.python.org/3/library/csv.html#csv-fmt-params

## Create Table of CMA number, area name, state
* note CMA001 - CMA306 are MSA (Metropolitan Service Areas)
* CMA307 onwards are RSA (Rural Service Areas)

In [87]:
path_to_csv_files = "data-cache/"

path_to_dropdown = ("%s/cma-dropdown_list.csv" % path_to_csv_files)
path_to_cmacnty = ("%s/cmacnty1990.xls" % path_to_csv_files)

In [82]:
# import cma_dropdown from csv
with open(path_to_dropdown, newline='\n') as f:
    reader = csv.reader(f, delimiter='-')
    m_dropdown = list(reader)

In [88]:
# split cma_no into first col
dropdown = []
for row in m_dropdown[1:]:
    dropdown.append([row[0].strip(), '-'.join(row[1:]).strip()])
dropdown[:10]

[['CMA001', 'New York, NY-NJ/Nassau-Suffolk'],
 ['CMA002', 'Los Angeles-Long Beach/Anaheim'],
 ['CMA003', 'Chicago, IL'],
 ['CMA004', 'Philadelphia, PA'],
 ['CMA005', 'Detroit/Ann Arbor, MI'],
 ['CMA006', 'Boston-Lowell-Brockton-Lawrence-Haverhill, MA-NH'],
 ['CMA007', 'San Francisco-Oakland, CA'],
 ['CMA008', 'Washington, DC-MD-VA'],
 ['CMA009', 'Dallas-Fort Worth, TX'],
 ['CMA010', 'Houston, TX']]

In [84]:
# check striped line matches original dropdown item

with open(path_to_dropdown, "r") as f:
    test_line = f.readlines()[1].strip()
    
assert(test_line == ' - '.join(dropdown[0]))

In [43]:
# add MSA/RSA variable at i=306 

for row in dropdown[:306]:
    row.append('MSA')
for row in dropdown[306:]:
    row.append('RSA')

In [44]:
# check RSA strings don't have any state codes
pattern_statecode = '[A-Z]{2}'

re_bool_statecode = []

for row in dropdown[306:]:
    string = row[1]
    re_bool_statecode.append(bool(re.search(pattern_statecode,string)))

assert(sum(re_bool_statecode) == 0)

In [45]:
# extract and append state codes for some MSAs

for row in dropdown:
    string = row[1]
    row.append(re.findall(pattern_statecode, string))        

In [46]:
# check MSA strings don't have any state numbers

pattern_statenum = '\d'
re_bool_statenum = []

for row in dropdown[:306]:
    string = row[1]
    re_bool_statenum.append(
        bool(re.search(pattern_statenum,string))
    )

assert(sum(re_bool_statenum) == 0)

In [47]:
# extract state name for RSAs

pattern = '([A-Za-z]+)(?:\s)(\d+)(?: - )(\D+)'

for row in dropdown:
    string = row[1]
    m = re.search(pattern, string)
    if m:
        row.append(list(m.groups()))
    else:
        row.append([])


In [48]:
# flatten last 

In [49]:
dropdown_cols = ['cma_code', 'cma_dscr', 'area_type', 'msa_state', 'rsa_dscr']
dropdown_df = pd.DataFrame(dropdown, columns=dropdown_cols)

In [50]:
# extract number component of CMA code

dropdown_df.insert(0, 'CMA', dropdown_df['cma_code'].apply(lambda x: x[3:]))

In [51]:
dropdown_4merge = dropdown_df.drop(['cma_code', 'msa_state', 'rsa_dscr'], 1)

### join cma-cnty with cma-dropdown

In [89]:
# import cma to county lookup table
# downloaded from https://www.cellularmaps.com/fcc_markets.shtml

cnty_df = pd.read_excel(path_to_cmacnty)

In [53]:
# pad CMA to string of length 3 for join with cma_lookup
cnty_df['CMA'] = cnty_df['CMA'].apply(lambda x: str(x).zfill(3))

In [54]:
# left merge cma_cnty info with dropdown descriptions using CMA

cnty_dropdown = cnty_df.merge(dropdown_4merge, on='CMA')

In [55]:
# 
cnty_dropdown.sort_values(by='CMA')

# how many states in each CMA
gb_CMA = cnty_dropdown.groupby('CMA')
gb_CMA.nunique()

# how many CMAs in particular state
gb_state = cnty_dropdown.groupby('State')
gb_state.nunique()

,County,State,FIPS,CMA,cma_dscr,area_type
State,,,,,,
AK,25,1,25,4,4,2
AL,67,1,67,18,18,2
AR,75,1,75,18,18,2
AS,5,1,5,1,1,1
AZ,15,1,15,8,8,2
CA,58,1,58,31,31,2
CO,63,1,63,14,14,2
CT,8,1,8,6,6,2
DC,1,1,1,1,1,1


## Merge dropdown & county info with scraped callsigns

### Import and join amend searches from csv

In [11]:
amend_files = os.listdir('search-cache/csv/')[1:]
print(amend_files)

['amend_CMA318_CMA722.csv', 'lt_10_CMA001_CMA734.csv', 'amend_CMA002_CMA317.csv']


In [17]:
search_cols = ['cma_no', 'callsign_leaseID', 'FRN_name', 'FRN_no', 'radio_service', 'status', 'date_expr' ]
combined = []

for f in amend_files:
    with open(('search-cache/csv/%s' % f), 'r', newline='') as table:
        reader = csv.reader(table)
        combined += list(reader)

pd.DataFrame(combined, columns=search_cols)

,cma_no,callsign_leaseID,FRN_name,FRN_no,radio_service,status,date_expr
0,CMA318,KNKN268,AT&T Mobility Spectrum LLC,0014980726,CL,Active,10/01/2020
1,CMA318,KNKQ365,Verizon Wireless (VAW) LLC,0003800307,CL,Active,10/01/2022
2,CMA318,KNKR314,ALLTEL Corporation,0002942159,CL,Active,01/19/2029
3,CMA318,L000022707,AT&T Mobility Spectrum LLC,0014980726,WY,Expired,09/22/2016
4,CMA318,L000022708,AT&T Mobility Spectrum LLC,0014980726,WY,Canceled,09/22/2017
5,CMA318,WPQL769,"Dobson Cellular Systems, Inc.",,CL,Expired,11/23/2000
6,CMA318,WPWV241,AT&T Mobility Spectrum LLC,0014980726,WZ,Active,06/13/2019
7,CMA318,WQGD712,AT&T Mobility Spectrum LLC,0014980726,AW,Active,12/18/2021
8,CMA318,WQJQ674,"Data-Max Wireless, LLC",0017166422,WY,Active,06/13/2019
9,CMA318,WQKQ560,"Tisdale Telephone Company, LLC",0018115550,CL,Terminated,08/12/2019


In [33]:
print("length of combined file", len(pd.DataFrame(combined)))
combined_df = (pd.DataFrame(combined, columns=search_cols)
               .drop_duplicates()
               .sort_values(by='cma_no')
              )
print("length of combined unique df", len(combined_df))

length of combined file 9327
length of combined unique df 6947


In [34]:
# save combined df to data-cache

combined_df.to_csv(path_or_buf='data-cache/cma_search_results_ALL.csv')

In [39]:
# filter for Cellular Licenses only
is_CL = (combined_df['radio_service'] == 'CL')
CL_df = combined_df[is_CL]

# save cellular-cma list to data-cache

CL_df.to_csv(path_or_buf='data-cache/cma_search_results_CL-only.csv')


# extract number component of CMA code
CL_df.insert(0,
             'CMA',
             CL_df['cma_no'].apply(lambda x: x[3:]))

# create look up table
CLsearch_4merge = CL_df.filter(['CMA', 'callsign_leaseID'])

In [66]:
# merge table of cma+counties with callsigns

callsigns_loc = CLsearch_4merge.merge(cnty_dropdown, on='CMA')

In [67]:
callsigns_loc.head()

,CMA,callsign_leaseID,County,State,FIPS,cma_dscr,area_type
0,001,KNKA206,Bergen,NJ,34003,"New York, NY-NJ/Nassau-Suffolk",MSA
1,001,KNKA206,Essex,NJ,34013,"New York, NY-NJ/Nassau-Suffolk",MSA
2,001,KNKA206,Hudson,NJ,34017,"New York, NY-NJ/Nassau-Suffolk",MSA
3,001,KNKA206,Morris,NJ,34027,"New York, NY-NJ/Nassau-Suffolk",MSA
4,001,KNKA206,Passaic,NJ,34031,"New York, NY-NJ/Nassau-Suffolk",MSA


In [73]:
cma_2_callsign = (callsigns_cma.groupby(['CMA', 'callsign_leaseID']).count()
                  .reset_index())[['CMA','callsign_leaseID']]

cma_2_callsign

,CMA,callsign_leaseID
0,001,KNKA206
1,001,KNKA310
2,002,KNKA209
3,002,KNKA351
4,002,KNKR294
5,002,KNKR313
6,002,L000015067
7,002,WPOI449
8,003,KNKA200
9,003,KNKA254


In [75]:
# cache to csv

callsigns_loc.to_csv('data-cache/cma_callsign_county.csv')

cma_2_callsign.to_csv('data-cache/cma_callsign_lookup.csv')